In [1]:
import pandas as pd 
import geopandas as gp
from shapely.ops import nearest_points
from shapely.geometry import Point
import numpy as np
from scipy.spatial import cKDTree

In [2]:
path = r'/home/semipro321/Documents/Workspace/Geospatial-Analysis-on-House-Price-Prediction/Inputs/'

In [3]:
df_house = pd.read_csv(path + 'House/house_with_neigh.csv')
df_school = pd.read_csv(path + 'City/Schools_with_neigh.csv')
gpf_house = gp.GeoDataFrame(df_house, crs="EPSG:4326", geometry=gp.points_from_xy(df_house.long,df_house.lat))
gpf_school= gp.GeoDataFrame(df_school, crs="EPSG:4326", geometry=gp.points_from_xy(df_school.LONGITUDE,df_school.LATITUDE))


In [4]:
def ckdnearest(gdA, gdB, distcol):
    gdA['geometry'] = gdA.geometry.to_crs('EPSG:5234')
    gdB['geometry'] = gdB.geometry.to_crs('EPSG:5234')

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    #If you want other features from B
    #gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            #If you want other features from B
            #gdB_nearest,
            pd.Series(dist, name=distcol)
        ], 
        axis=1)

    return gdf

In [5]:
gpf_house = ckdnearest(gpf_house, gpf_school, 'dist_nearest_school' )

In [6]:
df_ttc = pd.read_csv(path + 'Transit/Stops.csv')
gpf_ttc = gp.GeoDataFrame(df_ttc, crs="EPSG:4326", geometry=gp.points_from_xy(df_ttc.stop_lon,df_ttc.stop_lat))
gpf_house = ckdnearest(gpf_house, gpf_ttc, 'dist_nearest_ttc')

In [7]:
def ckdprox(gdA, gdB, distcol, meters):
    gdA['geometry'] = gdA.geometry.to_crs('EPSG:5234')
    gdB['geometry'] = gdB.geometry.to_crs('EPSG:5234')

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))   
    btree = cKDTree(nB)
    x = btree.query_ball_point(nA, meters)
    #If you want other features from B
    #gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            #If you want other features from B
            #gdB_nearest,
            pd.Series(x, name=distcol)
        ], 
        axis=1)
    gdf[distcol] = gdf[distcol].apply(lambda x: len(x))

    return gdf

In [8]:
df_crime = pd.read_csv(path + 'Crime/crime_transformed_030521.csv')
gpf_crime = gp.GeoDataFrame(df_crime, crs="EPSG:4326", geometry=gp.points_from_xy(df_crime.Long ,df_crime.Lat))

In [9]:
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Assault'] == 1) &( gpf_crime['datetime'] >= '2014-01-01') & (gpf_crime['datetime'] <= '2014-12-31')], "Assault_2014_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Assault'] == 1) &( gpf_crime['datetime'] >= '2015-01-01') & (gpf_crime['datetime'] <= '2015-12-31')], "Assault_2015_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Assault'] == 1) &( gpf_crime['datetime'] >= '2016-01-01') & (gpf_crime['datetime'] <= '2016-12-31')], "Assault_2016_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Assault'] == 1) &( gpf_crime['datetime'] >= '2017-01-01') & (gpf_crime['datetime'] <= '2017-12-31')], "Assault_2017_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Assault'] == 1) &( gpf_crime['datetime'] >= '2018-01-01') & (gpf_crime['datetime'] <= '2018-12-31')], "Assault_2018_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Assault'] == 1) &( gpf_crime['datetime'] >= '2019-01-01') & (gpf_crime['datetime'] <= '2019-12-31')], "Assault_2019_500m", 500)

In [10]:
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Auto Theft'] == 1) &( gpf_crime['datetime'] >= '2014-01-01') & (gpf_crime['datetime'] <= '2014-12-31')], "Auto Theft_2014_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Auto Theft'] == 1) &( gpf_crime['datetime'] >= '2015-01-01') & (gpf_crime['datetime'] <= '2015-12-31')], "Auto Theft_2015_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Auto Theft'] == 1) &( gpf_crime['datetime'] >= '2016-01-01') & (gpf_crime['datetime'] <= '2016-12-31')], "Auto Theft_2016_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Auto Theft'] == 1) &( gpf_crime['datetime'] >= '2017-01-01') & (gpf_crime['datetime'] <= '2017-12-31')], "Auto Theft_2017_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Auto Theft'] == 1) &( gpf_crime['datetime'] >= '2018-01-01') & (gpf_crime['datetime'] <= '2018-12-31')], "Auto Theft_2018_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Auto Theft'] == 1) &( gpf_crime['datetime'] >= '2019-01-01') & (gpf_crime['datetime'] <= '2019-12-31')], "Auto Theft_2019_500m", 500)

In [11]:
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Break and Enter'] == 1) &( gpf_crime['datetime'] >= '2014-01-01') & (gpf_crime['datetime'] <= '2014-12-31')], "Break and Enter_2014_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Break and Enter'] == 1) &( gpf_crime['datetime'] >= '2015-01-01') & (gpf_crime['datetime'] <= '2015-12-31')], "Break and Enter_2015_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Break and Enter'] == 1) &( gpf_crime['datetime'] >= '2016-01-01') & (gpf_crime['datetime'] <= '2016-12-31')], "Break and Enter_2016_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Break and Enter'] == 1) &( gpf_crime['datetime'] >= '2017-01-01') & (gpf_crime['datetime'] <= '2017-12-31')], "Break and Enter_2017_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Break and Enter'] == 1) &( gpf_crime['datetime'] >= '2018-01-01') & (gpf_crime['datetime'] <= '2018-12-31')], "Break and Enter_2018_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Break and Enter'] == 1) &( gpf_crime['datetime'] >= '2019-01-01') & (gpf_crime['datetime'] <= '2019-12-31')], "Break and Enter_2019_500m", 500)

In [12]:
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Robbery'] == 1) &( gpf_crime['datetime'] >= '2014-01-01') & (gpf_crime['datetime'] <= '2014-12-31')], "Robbery_2014_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Robbery'] == 1) &( gpf_crime['datetime'] >= '2015-01-01') & (gpf_crime['datetime'] <= '2015-12-31')], "Robbery_2015_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Robbery'] == 1) &( gpf_crime['datetime'] >= '2016-01-01') & (gpf_crime['datetime'] <= '2016-12-31')], "Robbery_2016_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Robbery'] == 1) &( gpf_crime['datetime'] >= '2017-01-01') & (gpf_crime['datetime'] <= '2017-12-31')], "Robbery_2017_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Robbery'] == 1) &( gpf_crime['datetime'] >= '2018-01-01') & (gpf_crime['datetime'] <= '2018-12-31')], "Robbery_2018_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Robbery'] == 1) &( gpf_crime['datetime'] >= '2019-01-01') & (gpf_crime['datetime'] <= '2019-12-31')], "Robbery_2019_500m", 500)

In [13]:
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Theft Over'] == 1) &( gpf_crime['datetime'] >= '2014-01-01') & (gpf_crime['datetime'] <= '2014-12-31')], "Theft Over_2014_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Theft Over'] == 1) &( gpf_crime['datetime'] >= '2015-01-01') & (gpf_crime['datetime'] <= '2015-12-31')], "Theft Over_2015_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Theft Over'] == 1) &( gpf_crime['datetime'] >= '2016-01-01') & (gpf_crime['datetime'] <= '2016-12-31')], "Theft Over_2016_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Theft Over'] == 1) &( gpf_crime['datetime'] >= '2017-01-01') & (gpf_crime['datetime'] <= '2017-12-31')], "Theft Over_2017_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Theft Over'] == 1) &( gpf_crime['datetime'] >= '2018-01-01') & (gpf_crime['datetime'] <= '2018-12-31')], "Theft Over_2018_500m", 500)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Theft Over'] == 1) &( gpf_crime['datetime'] >= '2019-01-01') & (gpf_crime['datetime'] <= '2019-12-31')], "Theft Over_2019_500m", 500)

In [14]:
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Assault'] == 1) &( gpf_crime['datetime'] >= '2014-01-01') & (gpf_crime['datetime'] <= '2014-12-31')], "Assault_2014_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Assault'] == 1) &( gpf_crime['datetime'] >= '2015-01-01') & (gpf_crime['datetime'] <= '2015-12-31')], "Assault_2015_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Assault'] == 1) &( gpf_crime['datetime'] >= '2016-01-01') & (gpf_crime['datetime'] <= '2016-12-31')], "Assault_2016_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Assault'] == 1) &( gpf_crime['datetime'] >= '2017-01-01') & (gpf_crime['datetime'] <= '2017-12-31')], "Assault_2017_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Assault'] == 1) &( gpf_crime['datetime'] >= '2018-01-01') & (gpf_crime['datetime'] <= '2018-12-31')], "Assault_2018_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Assault'] == 1) &( gpf_crime['datetime'] >= '2019-01-01') & (gpf_crime['datetime'] <= '2019-12-31')], "Assault_2019_1000m", 1000)


gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Auto Theft'] == 1) &( gpf_crime['datetime'] >= '2014-01-01') & (gpf_crime['datetime'] <= '2014-12-31')], "Auto Theft_2014_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Auto Theft'] == 1) &( gpf_crime['datetime'] >= '2015-01-01') & (gpf_crime['datetime'] <= '2015-12-31')], "Auto Theft_2015_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Auto Theft'] == 1) &( gpf_crime['datetime'] >= '2016-01-01') & (gpf_crime['datetime'] <= '2016-12-31')], "Auto Theft_2016_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Auto Theft'] == 1) &( gpf_crime['datetime'] >= '2017-01-01') & (gpf_crime['datetime'] <= '2017-12-31')], "Auto Theft_2017_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Auto Theft'] == 1) &( gpf_crime['datetime'] >= '2018-01-01') & (gpf_crime['datetime'] <= '2018-12-31')], "Auto Theft_2018_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Auto Theft'] == 1) &( gpf_crime['datetime'] >= '2019-01-01') & (gpf_crime['datetime'] <= '2019-12-31')], "Auto Theft_2019_1000m", 1000)



gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Break and Enter'] == 1) &( gpf_crime['datetime'] >= '2014-01-01') & (gpf_crime['datetime'] <= '2014-12-31')], "Break and Enter_2014_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Break and Enter'] == 1) &( gpf_crime['datetime'] >= '2015-01-01') & (gpf_crime['datetime'] <= '2015-12-31')], "Break and Enter_2015_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Break and Enter'] == 1) &( gpf_crime['datetime'] >= '2016-01-01') & (gpf_crime['datetime'] <= '2016-12-31')], "Break and Enter_2016_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Break and Enter'] == 1) &( gpf_crime['datetime'] >= '2017-01-01') & (gpf_crime['datetime'] <= '2017-12-31')], "Break and Enter_2017_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Break and Enter'] == 1) &( gpf_crime['datetime'] >= '2018-01-01') & (gpf_crime['datetime'] <= '2018-12-31')], "Break and Enter_2018_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Break and Enter'] == 1) &( gpf_crime['datetime'] >= '2019-01-01') & (gpf_crime['datetime'] <= '2019-12-31')], "Break and Enter_2019_1000m", 1000)



gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Robbery'] == 1) &( gpf_crime['datetime'] >= '2014-01-01') & (gpf_crime['datetime'] <= '2014-12-31')], "Robbery_2014_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Robbery'] == 1) &( gpf_crime['datetime'] >= '2015-01-01') & (gpf_crime['datetime'] <= '2015-12-31')], "Robbery_2015_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Robbery'] == 1) &( gpf_crime['datetime'] >= '2016-01-01') & (gpf_crime['datetime'] <= '2016-12-31')], "Robbery_2016_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Robbery'] == 1) &( gpf_crime['datetime'] >= '2017-01-01') & (gpf_crime['datetime'] <= '2017-12-31')], "Robbery_2017_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Robbery'] == 1) &( gpf_crime['datetime'] >= '2018-01-01') & (gpf_crime['datetime'] <= '2018-12-31')], "Robbery_2018_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Robbery'] == 1) &( gpf_crime['datetime'] >= '2019-01-01') & (gpf_crime['datetime'] <= '2019-12-31')], "Robbery_2019_1000m", 1000)



gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Theft Over'] == 1) &( gpf_crime['datetime'] >= '2014-01-01') & (gpf_crime['datetime'] <= '2014-12-31')], "Theft Over_2014_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Theft Over'] == 1) &( gpf_crime['datetime'] >= '2015-01-01') & (gpf_crime['datetime'] <= '2015-12-31')], "Theft Over_2015_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Theft Over'] == 1) &( gpf_crime['datetime'] >= '2016-01-01') & (gpf_crime['datetime'] <= '2016-12-31')], "Theft Over_2016_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Theft Over'] == 1) &( gpf_crime['datetime'] >= '2017-01-01') & (gpf_crime['datetime'] <= '2017-12-31')], "Theft Over_2017_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Theft Over'] == 1) &( gpf_crime['datetime'] >= '2018-01-01') & (gpf_crime['datetime'] <= '2018-12-31')], "Theft Over_2018_1000m", 1000)
gpf_house = ckdprox(gpf_house, gpf_crime[(gpf_crime['Category__Theft Over'] == 1) &( gpf_crime['datetime'] >= '2019-01-01') & (gpf_crime['datetime'] <= '2019-12-31')], "Theft Over_2019_1000m", 1000)



In [16]:
col_sub1000 = []
for i in range(2014, 2020):
    col_sub1000.append(f'Assault_{i}_1000m')
    col_sub1000.append(f'Auto Theft_{i}_1000m')
    col_sub1000.append(f'Break and Enter_{i}_1000m')
    col_sub1000.append(f'Robbery_{i}_1000m')
    col_sub1000.append(f'Theft Over_{i}_1000m')

col_sub500 = []
for i in range(2014, 2020):
    col_sub500.append(f'Assault_{i}_500m')
    col_sub500.append(f'Auto Theft_{i}_500m')
    col_sub500.append(f'Break and Enter_{i}_500m')
    col_sub500.append(f'Robbery_{i}_500m')
    col_sub500.append(f'Theft Over_{i}_500m')




In [17]:
for i, j in zip(col_sub1000,col_sub500):
    gpf_house[i] = gpf_house[i] - gpf_house[j] 

In [18]:
gpf_house

,lat,long,sqft,parking,mean_district_income,bedrooms_bg,bedrooms_ag,bathrooms,final_price,type_Att/Row/Twnhouse,...,Robbery_2016_1000m,Robbery_2017_1000m,Robbery_2018_1000m,Robbery_2019_1000m,Theft Over_2014_1000m,Theft Over_2015_1000m,Theft Over_2016_1000m,Theft Over_2017_1000m,Theft Over_2018_1000m,Theft Over_2019_1000m
0,43.661896,-79.385748,850.0,1,56526,1,2,2,855000,0,...,133,132,166,134,30,42,29,38,46,48
1,43.646982,-79.530356,NaN,6,52787,0,3,2,885000,0,...,9,12,11,5,3,0,1,2,3,6
2,43.708472,-79.397498,550.0,0,57039,0,1,1,550000,0,...,9,6,13,7,6,1,4,3,6,7
3,43.638399,-79.414448,650.0,1,70623,1,1,1,665000,0,...,13,8,8,15,4,12,14,7,11,20
4,43.597096,-79.509612,NaN,1,44101,0,2,2,825513,0,...,8,7,6,9,0,3,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15229,43.648317,-79.393065,750.0,1,70600,0,2,2,785000,0,...,60,54,59,53,41,41,39,40,46,66
15230,43.706338,-79.262789,3250.0,2,36232,0,5,5,1025000,0,...,11,12,5,9,0,2,1,1,4,2
15231,43.788258,-79.160342,NaN,1,54045,0,3,3,681000,1,...,3,3,0,1,1,0,1,2,4,1
15232,43.660846,-79.378756,550.0,0,53583,0,1,1,590000,0,...,170,202,228,213,38,37,43,30,62,59
